In [22]:
import torch
import math

In [4]:
batch_size = 10

tensor1 = torch.randn(batch_size, 5)
tensor2 = torch.randn(batch_size, 5)
#torch.matmul(tensor1, tensor2).size()

In [5]:
query_layer = tensor1
key_layer = tensor2

scores = torch.matmul(query_layer, key_layer.transpose(-1, -2))

In [7]:
scores.size()

torch.Size([10, 10])

In [11]:
class OcrPtrNet(torch.nn.Module):
    def __init__(self, hidden_size, query_key_size=None):
        super().__init__()

        if query_key_size is None:
            query_key_size = hidden_size
        self.hidden_size = hidden_size
        self.query_key_size = query_key_size

        self.query = torch.nn.Linear(hidden_size, query_key_size)
        self.key = torch.nn.Linear(hidden_size, query_key_size)

    def forward(self, query_inputs, key_inputs, attention_mask):
        extended_attention_mask = (1.0 - attention_mask) * -10000.0
        assert extended_attention_mask.dim() == 2
        extended_attention_mask = extended_attention_mask.unsqueeze(1)

        query_layer = self.query(query_inputs)
        if query_layer.dim() == 2:
            query_layer = query_layer.unsqueeze(1)
            squeeze_result = True
        else:
            squeeze_result = False
        key_layer = self.key(key_inputs)

        scores = torch.matmul(query_layer, key_layer.transpose(-1, -2))
        scores = scores / math.sqrt(self.query_key_size)
        scores = scores + extended_attention_mask
        if squeeze_result:
            scores = scores.squeeze(1)

        return scores


In [13]:
hidden_size = 10
q_k_size = 7

ocr_ptr_net = OcrPtrNet(hidden_size=hidden_size, query_key_size=q_k_size)

In [29]:

q_inputs = torch.randn(batch_size, 15, 10)
k_inputs = torch.randn(batch_size, 15, 10)
mmt_dec_output = q_inputs
mmt_ocr_output = k_inputs
ocr_mask = torch.randn(10, 15)

dynamic_ocr_scores = ocr_ptr_net(mmt_dec_output, mmt_ocr_output, ocr_mask)

In [27]:
dynamic_ocr_scores.size()

torch.Size([10, 15, 15])